Финальное задание.
1.  Использовать можно весь набор данных Jester DataSet, но сохранив разбивку на выборку для обучения и для теста (id участников и шуток сохранены в github (https://github.com/NameArtem/recommendation_in_practice_tutorial/tree/main/code/BaseLine))

2.  Разрешено использовать новые варианты алгоритмов, но желательно не уходить далеко в «сложные нейронные сети» и не использовать архитектуры, которые вы не понимаете (случайные архитектуры нейронных сетей из интернета).
Лучшим вариантом является гибридная рекомендательная система с разными эвристиками до или после скоринга + ранжировщик результатов. Это покажет изучение темы и самого набора данных (помните про ответственную разработку своих решений).

3.  Результатом вашей рекомендательной системы должна быть последовательность (список в списках) из двух результатов: [[топ рекомендация], [ранжированная последовательность из 10 рекомендаций]].

4.  Позиция 1 будет проверена по метрике RMSE, ранжированная последовательность будет оценена по MAP@10 

5.  Основная задача сохранить метрику, которую вы получили на Kaggle лидерборде, но упор делаем на метрики: diversity, serendipity, novelty, coverage.
Помните, что необходимо реализовать баланс, где хороший топ будет интересен и в наборе.

6.  Результат работы желательно встроить в контейнер, чтобы его можно было бы запустить автопроверкой.
Контейнер должен получать данные, трансформировать их в готовый pandas DataFrame для работы, скорить переданных клиентов и возвращать результат в связке: клиент и результат из п.3 (см.выше).
Контейнер уже реализован, ваша модель должна быть его основным механизмом. Пример реализации: https://github.com/TripleAVerAlpha/docker

7.  Если не удаётся запустить контейнер или его создать, то на вопросы ответит Золотых Александр.
Учитывайте, что контейнер, может быть не реализован (т.к. понимание, что сложно и важнее реализовать модель), но сама модель должна запускаться.

8.  Результат прошу загрузить на GitHub, а ссылку передать через форму https://forms.gle/Ay2b9z7nuhtgPBkFA

оснвной датасет - https://drive.google.com/file/d/1lK62v94zu246tJfvaxgjP_QBswwXHvD4/view?usp=sharing
датасет с текстами шуток - https://drive.google.com/file/d/1NcM0Qk1MUT-NuTfSfwkIfdEp1DXDAIr4/view?usp=drivesdk

* данные должны выходить из контейнера в таком формате - csv в виде [UID, {JID: Rating}, [все JID]].

In [26]:
! unzip -f recsys-in-practice.zip

Archive:  recsys-in-practice.zip


In [27]:
! unzip -f jester_dataset_1_joke_texts.zip

Archive:  jester_dataset_1_joke_texts.zip


In [28]:
!python3 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


Устанавливаем библиотеки

In [29]:
!pip install catboost 
!pip install numpy
!pip install scikit-learn
!pip install nltk
!pip install xgboost
!pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Импортируем бибилиотеки

In [30]:
import pandas as pd
import numpy as np
# sklearn
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from catboost import Pool, CatBoostRegressor, cv
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# surprise
from surprise import Dataset, Reader, KNNWithMeans, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import KFold
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise.accuracy import rmse
from surprise import accuracy
import os
import re
from catboost import CatBoostRegressor, Pool
import catboost

In [31]:
# образец предсказания
df_sample = pd.read_csv('sample_submission.csv')
# тестовая выборка
df_test = pd.read_csv('test_joke_df_nofactrating.csv')
# обучающая выборка
df_train = pd.read_csv('train_joke_df.csv')

In [32]:
# Указать путь к папке с файлами
folder_path = "jokes"

# Получить список файлов в папке
file_list = os.listdir(folder_path)

Добавляем тексты шуток в таблицу

In [33]:
def add_joke_text_from_files(folder_path, df_train):
    # Получить список файлов с шутками из заданной папки
    file_list = os.listdir(folder_path)

    # Создать пустой список для хранения номеров шуток и текстов шуток
    joke_num_list = []
    text_list = []

    # Пройти по списку файлов и извлечь номера шуток и тексты шуток
    for file_name in file_list:
        # Проверить, что имя файла не является скрытым файлом
        if not file_name.startswith("."):
            # Извлечь номер шутки из имени файла с помощью регулярного выражения
            joke_num_match = re.findall(r"init(\d+)\.html", file_name)
            if joke_num_match:
                joke_num = int(joke_num_match[0])
                # Проверить, есть ли номер шутки в DataFrame df_train
                if joke_num in df_train["JID"].values:
                    # Извлечь текст шутки из файла
                    file_path = os.path.join(folder_path, file_name)
                    with open(file_path, "r", encoding="utf-8") as f:
                        file_content = f.read()
                        # Извлечь текст из HTML с помощью регулярного выражения
                        text = re.sub(r"<.*?>", "", file_content)
                        # Удалить символы переноса строки
                        text = text.replace("\n", "")
                        # Удалить "A Joke" и "Joke #"
                        text = re.sub(r"(Joke \d+)|(A Joke),?",
                                      "", text).strip()
                        text_list.append(text)
                        joke_num_list.append(joke_num)

    # Создать DataFrame с номерами шуток и текстами шуток
    jokes_df = pd.DataFrame({"JID": joke_num_list, "joke_text": text_list})

    # Объединить DataFrame df_train и jokes_df по столбцу JID
    df_train = pd.merge(df_train, jokes_df, on="JID", how="left")

    return df_train

In [34]:
data = add_joke_text_from_files('jokes', df_train)
data

,UID,JID,Rating,joke_text
0,18029,6,-1.26,Bill & Hillary are on a trip back to Arkansas....
1,3298,64,-4.17,What is the rallying cry of the International ...
2,3366,58,0.92,How many teddybears does it take to change a l...
3,12735,92,3.69,Early one morning a mother went to her sleepin...
4,11365,38,-6.60,"""May I take your order?"" the waiter asked. ""Ye..."
...,...,...,...,...
1448359,22604,26,2.82,A guy walks into a bar and sits down next to a...
1448360,22255,36,-1.94,"A guy walks into a bar, orders a beer and says..."
1448361,21056,40,-9.56,How many Irishmen does it take to change a lig...
1448362,12328,97,0.87,A teacher is explaining to her class how diffe...


Добавляем тексты шуток в тестовую таблицу

In [35]:
data_test = add_joke_text_from_files('jokes', df_test)
data_test.head(10)

,InteractionID,UID,JID,joke_text
0,0,11228,39,What is the difference between men and women:A...
1,1,21724,85,Q: How many Presidents does it take to screw i...
2,2,16782,56,A man and Cindy Crawford get stranded on a des...
3,3,12105,42,Two men are discussing the age old question: w...
4,4,14427,2,of 25 This couple had an excellent relations...
5,5,4436,13,of 20 They asked the Japanese visitor if th...
6,6,8311,45,A boy comes home from school and tells his mot...
7,7,20915,9,A country guy goes into a city bar that has a ...
8,8,2392,33,What do you call an American in the finals of ...
9,9,12458,15,of 20 Q: What did the blind person say when...


Проверяем данные различными срезами

In [36]:
data.query('JID==7')

,UID,JID,Rating,joke_text
139,14903,7,-8.06,How many feminists does it take to screw in a ...
546,24253,7,-9.90,How many feminists does it take to screw in a ...
598,8522,7,-7.57,How many feminists does it take to screw in a ...
656,3513,7,-2.82,How many feminists does it take to screw in a ...
802,5814,7,7.28,How many feminists does it take to screw in a ...
...,...,...,...,...
1448135,9056,7,-5.97,How many feminists does it take to screw in a ...
1448258,7633,7,-4.22,How many feminists does it take to screw in a ...
1448277,16620,7,-7.23,How many feminists does it take to screw in a ...
1448303,24542,7,-4.66,How many feminists does it take to screw in a ...


In [37]:
data_test.query('JID==7')

,InteractionID,UID,JID,joke_text
19,19,10762,7,How many feminists does it take to screw in a ...
185,185,21704,7,How many feminists does it take to screw in a ...
430,430,13926,7,How many feminists does it take to screw in a ...
496,496,3826,7,How many feminists does it take to screw in a ...
669,669,11944,7,How many feminists does it take to screw in a ...
...,...,...,...,...
361799,361799,3992,7,How many feminists does it take to screw in a ...
361847,361847,9279,7,How many feminists does it take to screw in a ...
361892,361892,16161,7,How many feminists does it take to screw in a ...
361998,361998,8426,7,How many feminists does it take to screw in a ...


In [38]:
uid_grouped = data.groupby('UID')['JID'].agg('count')
uid_grouped.head(5)

UID
1    62
2    84
3    41
4    39
5    80
Name: JID, dtype: int64

In [39]:
data.groupby('JID')['Rating'].mean().sort_values(ascending=False).head(15)

JID
50    3.648577
89    3.623128
36    3.319388
27    3.207354
32    3.181644
62    3.000426
35    2.981776
29    2.953855
53    2.939566
72    2.830575
49    2.760112
54    2.753794
68    2.618187
69    2.582571
66    2.518668
Name: Rating, dtype: float64

In [40]:
uid_grouped = data.groupby('UID')
uid_grouped

In [41]:
max_ratings = data.groupby('UID')['Rating'].idxmax()
#получили таблицу с любимыми шутками для каждого пользователя
# Выбор строк с наивысшей оценкой для каждого пользователя
result = data.loc[max_ratings]
result

,UID,JID,Rating,joke_text
80144,1,29,9.13,An old Scotsmen is sitting with a younger Scot...
23332,2,10,9.22,"of 20 Two cannibals are eating a clown, one..."
506425,3,13,9.37,of 20 They asked the Japanese visitor if th...
709263,4,2,8.35,of 25 This couple had an excellent relations...
876585,5,53,8.93,One Sunday morning William burst into the livi...
...,...,...,...,...
922980,24979,64,9.27,What is the rallying cry of the International ...
847206,24980,95,9.22,"Just a thought ..Before criticizing someone, w..."
861452,24981,51,7.38,Did you hear that Clinton has announced there ...
642448,24982,29,8.30,An old Scotsmen is sitting with a younger Scot...


In [42]:
len(data.UID.unique())

24983

In [43]:
train = data.sample(frac=0.8).copy()
train

,UID,JID,Rating,joke_text
668296,24599,89,7.86,A radio conversation of a US naval ship with C...
644217,11884,3,-3.01,Q. What's 200 feet long and has 4 teeth? A. Th...
928132,23705,33,0.10,What do you call an American in the finals of ...
1147008,3256,43,-8.98,Arnold Swartzeneger and Sylvester Stallone are...
1189487,1198,96,4.32,Two attorneys went into a diner and ordered tw...
...,...,...,...,...
130763,16229,22,3.98,A duck walks into a pharmacy and asks for a co...
1145038,5396,50,7.91,A guy goes into confession and says to the pri...
323356,2312,45,1.55,A boy comes home from school and tells his mot...
501517,14555,1,-6.75,of 25 A man visits the doctor. The doctor say...


In [44]:
val = data[~data.index.isin(train.index)].copy()


### CAT BOOST

Сначала обрабатывала текст с помощью TF_IDF и NFM но обработка идет очень долго и при merge данных отваливается ядро, решила прогнозировать просто на основе модели, которая может принимать текст

In [45]:
X_col = ['joke_text', 'UID']
y_col = ['Rating']
text_features = ['joke_text']

In [46]:
model = catboost.CatBoostRegressor(task_type="CPU", text_features=text_features,  # loss_function='RMSE',
                                   # iterations=1500, learning_rate=0.02, depth=6)
                                   iterations=1000,
                                   learning_rate=0.1,
                                   depth=4,
                                   l2_leaf_reg=1,
                                   bagging_temperature=0.8,
                                   random_strength=0.5,
                                   border_count=100,
                                   bootstrap_type='Bayesian', verbose=150)

In [47]:
model.fit(train[X_col], train[y_col],
          eval_set=(val[X_col], val[y_col]))

0:	learn: 5.2125272	test: 5.2095529	best: 5.2095529 (0)	total: 541ms	remaining: 9m
150:	learn: 4.9840452	test: 4.9789523	best: 4.9789523 (150)	total: 1m 5s	remaining: 6m 8s
300:	learn: 4.9777453	test: 4.9727431	best: 4.9727431 (300)	total: 2m 11s	remaining: 5m 5s
450:	learn: 4.9754256	test: 4.9710410	best: 4.9710410 (450)	total: 3m 20s	remaining: 4m 3s
600:	learn: 4.9741115	test: 4.9702622	best: 4.9702622 (600)	total: 4m 33s	remaining: 3m 1s
750:	learn: 4.9731402	test: 4.9697817	best: 4.9697817 (750)	total: 5m 44s	remaining: 1m 54s
900:	learn: 4.9723772	test: 4.9694581	best: 4.9694581 (900)	total: 6m 53s	remaining: 45.5s
999:	learn: 4.9720247	test: 4.9693776	best: 4.9693776 (999)	total: 7m 38s	remaining: 0us

bestTest = 4.969377612
bestIteration = 999



In [48]:
model_i= catboost.CatBoostRegressor(task_type="CPU", text_features=text_features,  # loss_function='RMSE',
                                   iterations=300, learning_rate=0.1, depth=4, verbose=30
                                   )

In [49]:
model_i.fit(result[X_col], result[y_col],
          eval_set=(val[X_col], val[y_col]))

0:	learn: 1.7406837	test: 8.6736092	best: 8.6736092 (0)	total: 28.6ms	remaining: 8.56s
30:	learn: 1.7331924	test: 8.6799339	best: 8.6736092 (0)	total: 632ms	remaining: 5.49s
60:	learn: 1.7304290	test: 8.6828576	best: 8.6736092 (0)	total: 1.22s	remaining: 4.77s
90:	learn: 1.7290700	test: 8.6856099	best: 8.6736092 (0)	total: 1.8s	remaining: 4.14s
120:	learn: 1.7255863	test: 8.6879579	best: 8.6736092 (0)	total: 2.4s	remaining: 3.55s
150:	learn: 1.7209446	test: 8.6892030	best: 8.6736092 (0)	total: 3.03s	remaining: 2.99s
180:	learn: 1.7171062	test: 8.6897389	best: 8.6736092 (0)	total: 3.66s	remaining: 2.41s
210:	learn: 1.7136114	test: 8.6911857	best: 8.6736092 (0)	total: 4.28s	remaining: 1.81s
240:	learn: 1.7106653	test: 8.6919212	best: 8.6736092 (0)	total: 4.91s	remaining: 1.2s
270:	learn: 1.7077504	test: 8.6927765	best: 8.6736092 (0)	total: 5.52s	remaining: 591ms
299:	learn: 1.7049962	test: 8.6933048	best: 8.6736092 (0)	total: 6.13s	remaining: 0us

bestTest = 8.673609177
bestIteration = 0

In [50]:
def recommend_items_for_user_i(UID):
    # Нахождение элементов, которые пользователь оценил в обучающем наборе данных
    user_rated_jokes = result[result['UID'] == UID]['JID'].unique()

    # Нахождение элементов, которые пользователь не оценил в обучающем наборе данных, но присутствуют в тестовом наборе данных
    items_to_recommend = np.setdiff1d(
        data_test['JID'].unique(), user_rated_jokes)

    # Создание DataFrame с элементами, которые пользователь не оценивал
    user_items = data_test[data_test['JID'].isin(
        items_to_recommend)].drop_duplicates(subset='JID').copy()
    user_items['UID'] = UID

    # # Получение предсказаний для элементов, которые пользователь еще не оценил
    predictions = model_i.predict(user_items[X_col])

    # # Создание DataFrame с предсказанными оценками
    user_items['Wei'] = predictions

    # # Сортировка элементов по предсказанной оценке
    recommended_items = user_items.groupby(
         'JID').max().sort_values(by='Wei', ascending=False)

    # # Получение топ-N рекомендаций для пользователя
    #top_n_recommendations = recommended_items.head(10).index.tolist()

    return recommended_items

In [51]:
recommend_items_for_user_i(45)

,InteractionID,UID,joke_text,Wei
JID,,,,
12,164,45,of 20 A guy stood over his tee shot for what...,7.805265
26,13,45,A guy walks into a bar and sits down next to a...,7.805265
1,125,45,of 25 A man visits the doctor. The doctor say...,7.796804
65,117,45,Two Rednecks were seated at the end of a bar w...,7.796804
75,66,45,Q: Do you know the difference between an intel...,7.796804
...,...,...,...,...
69,308,45,"This guys wife asks, ""Honey if I died would yo...",7.779130
53,62,45,One Sunday morning William burst into the livi...,7.779130
50,41,45,A guy goes into confession and says to the pri...,7.762425


In [52]:
from sklearn.metrics import mean_squared_error

# Получение предсказаний модели для валидационной выборки
predictions = model.predict(val[X_col])

# Вычисление RMSE
rmse = mean_squared_error(val[y_col], predictions, squared=False)

print("RMSE:", rmse)

RMSE: 4.969377605946551


In [53]:
data_test

,InteractionID,UID,JID,joke_text
0,0,11228,39,What is the difference between men and women:A...
1,1,21724,85,Q: How many Presidents does it take to screw i...
2,2,16782,56,A man and Cindy Crawford get stranded on a des...
3,3,12105,42,Two men are discussing the age old question: w...
4,4,14427,2,of 25 This couple had an excellent relations...
...,...,...,...,...
362086,362086,3085,66,"A lawyer opened the door of his BMW, when sudd..."
362087,362087,13765,31,President Clinton looks up from his desk in th...
362088,362088,10341,29,An old Scotsmen is sitting with a younger Scot...
362089,362089,3553,8,Q. Did you hear about the dyslexic devil worsh...


тестируем

In [54]:
def recommend_items_for_user(UID):
    # Нахождение элементов, которые пользователь оценил в обучающем наборе данных
    user_rated_jokes = train[train['UID'] == UID]['JID'].unique()

    # Нахождение элементов, которые пользователь не оценил в обучающем наборе данных, но присутствуют в тестовом наборе данных
    items_to_recommend = np.setdiff1d(
        data_test['JID'].unique(), user_rated_jokes)

    # Создание DataFrame с элементами, которые пользователь не оценивал
    user_items = data_test[data_test['JID'].isin(
        items_to_recommend)].drop_duplicates(subset='JID').copy()
    user_items['UID'] = UID

    # # Получение предсказаний для элементов, которые пользователь еще не оценил
    predictions = model.predict(user_items[X_col])

    # # Создание DataFrame с предсказанными оценками
    user_items['Rating'] = predictions

    # # Сортировка элементов по предсказанной оценке
    recommended_items = user_items.groupby(
         'JID').max().sort_values(by='Rating', ascending=False)

    # # Получение топ-N рекомендаций для пользователя
    top_n_recommendations = recommended_items.head(10).index.tolist()

    return top_n_recommendations

In [55]:
len(recommend_items_for_user(45))

10

In [56]:
recommend_items_for_user(45)

[89, 36, 49, 66, 76, 21, 68, 54, 91, 6]

In [57]:
recommend_items_for_user(45)

[89, 36, 49, 66, 76, 21, 68, 54, 91, 6]

### SURPRISE

Эта модель будет обучаться только на JID, UID, Rating в отличии от Cat Boost который обучался на тексте

In [58]:
# считываем данные из датасета
reader = Reader(rating_scale=(-10, 10))
data_s = Dataset.load_from_df(df_train[['UID', 'JID', 'Rating']], reader)

In [59]:
trainingSet = data_s.build_full_trainset()

In [60]:
algo = SVD(n_epochs=100, lr_all=0.01, reg_all=0.02, n_factors=300)

kf = KFold(n_splits=5)

for trainset, testset in kf.split(data_s):

    algo.fit(trainingSet)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.5570
RMSE: 0.5628
RMSE: 0.5644
RMSE: 0.5601


Напишем функцию, для применения модели на разных UID

In [ ]:
def recommend_items_for_user_s(UID):
    # Нахождение элементов, которые пользователь оценил в обучающем наборе данных
    user_rated_jokes = train[train['UID'] == UID]['JID'].unique()

    # Нахождение элементов, которые пользователь не оценил в обучающем наборе данных, но присутствуют в тестовом наборе данных
    items_to_recommend = np.setdiff1d(
        df_test['JID'].unique(), user_rated_jokes)

    user_items = df_test[df_test['JID'].isin(
        items_to_recommend)].drop_duplicates(subset='JID').copy()
    user_items['UID'] = UID

    # Добавление столбца "Rating" в user_items со значениями заглушками
    user_items['Rating'] = 0

    # Создание Dataset объекта из данных user_items
    dataset = Dataset.load_from_df(
        user_items[['UID', 'JID', 'Rating']], Reader())

    # Получение предсказаний для элементов, которые пользователь еще не оценил
    predictions = algo.test(dataset.build_full_trainset().build_testset())

    # Создание DataFrame с предсказанными оценками
    user_items['Rating'] = [pred.est for pred in predictions]

    # Сортировка элементов по предсказанной оценке
    recommended_items = user_items.groupby(
        'JID').max().sort_values(by='Rating', ascending=False)

    # Получение топ-N рекомендаций для пользователя
    top_n_recommendations = recommended_items.head(10).index.tolist()

    return top_n_recommendations

In [ ]:
# def recommend_items_for_user_s(UID):
#     # Нахождение элементов, которые пользователь оценил в обучающем наборе данных
#     user_rated_jokes = train[train['UID'] == UID]['JID'].unique()

#     # Нахождение элементов, которые пользователь не оценил в обучающем наборе данных, но присутствуют в тестовом наборе данных
#     items_to_recommend = np.setdiff1d(df_test['JID'].unique(), user_rated_jokes)

#     user_items = df_test[df_test['JID'].isin(items_to_recommend)].drop_duplicates(subset='JID').copy()
#     user_items['UID'] = UID


#     # Получение предсказаний для элементов, которые пользователь еще не оценил
#     predictions = data_test[['UID', 'JID']].apply(lambda x: algo.predict(x[0], x[1], verbose=False).est,
#                                                       axis = 1)

#     # Создание DataFrame с предсказанными оценками
#     user_items['Rating'] = predictions

#     # Сортировка элементов по предсказанной оценке
#     recommended_items = user_items.groupby('JID').max().sort_values(by='Rating', ascending=False)

#     # Получение топ-N рекомендаций для пользователя
#     top_n_recommendations = recommended_items.head(10).index.tolist()

#     return top_n_recommendations

Проверяем работы функций на разных UID

In [ ]:
recommend_items_for_user_s(45)

[30, 27, 56, 47, 34, 25, 54, 6, 66, 11]

In [ ]:
recommend_items_for_user(45)

[29, 27, 66, 76, 93, 54, 88, 56, 91, 6]

In [ ]:
print(recommend_items_for_user_s(1))
recommend_items_for_user(1)

[29, 27, 65, 89, 49, 31, 46, 93, 72, 91]


[89, 29, 27, 32, 49, 72, 65, 76, 93, 31]

In [ ]:
print(recommend_items_for_user_s(987654321))
recommend_items_for_user(987654321)

[50, 89, 27, 36, 62, 29, 35, 32, 53, 72]


[50, 89, 29, 72, 36, 32, 35, 27, 53, 93]

### Балансируем резульаты моделей

In [ ]:
def ensemble_recommendations(UID):
    # Получение рекомендаций от первой модели
    recommendations1 = recommend_items_for_user_s(UID)

    # Получение рекомендаций от второй модели
    recommendations2 = recommend_items_for_user(UID)

    # Получение шуток, которые рекомендуют обе модели
    # common_recommendations = set(recommendations1).intersection(recommendations2)
    common_recommendations = []
    [common_recommendations.append(
        x) for x in recommendations1 if x in recommendations2]

    return list(common_recommendations)

In [ ]:
print(recommend_items_for_user_s(1680))
recommend_items_for_user(1680)

[41, 61, 54, 53, 72, 76, 68, 63, 35, 89]


[89, 29, 35, 53, 54, 68, 72, 61, 76, 88]

если списко 10 шуток не полный, думаю заполнять его самыми популярными шутками - так как большинство пользователей все же имеют достаточно много рекомендованных шуток

In [ ]:
ensemble_recommendations(16)

[27, 21, 35, 29, 56]

In [ ]:
top_jokes = df_train.groupby(
    'JID')['Rating'].mean().sort_values(ascending=False)
top_jokes_list = top_jokes.head(10).index.tolist()
top_jokes_list

[50, 89, 36, 27, 32, 62, 35, 29, 53, 72]

In [ ]:
def ensemble_recommendations_f(UID):
    recommendations1 = recommend_items_for_user_s(UID)
    recommendations2 = recommend_items_for_user(UID)

    # Получение шуток, которые рекомендуют обе модели
    common_recommendations = []
    [common_recommendations.append(
        x) for x in recommendations1 if x in recommendations2]

    if len(common_recommendations) < 10:
        # Дополнение списка до 10 элементов
        additional_recommendations = top_jokes_list[:10]
        while len(common_recommendations) < 10:
            common_recommendations.append(additional_recommendations.pop(0))

    return common_recommendations

In [ ]:
ensemble_recommendations_f(1680)

[61, 54, 53, 72, 76, 68, 35, 89, 50, 89]

## ВЫВОД РЕЗУЛЬТАТА - наиболее рекомендованная шутка и топ 10 шуток

In [ ]:
def final_func (UID):
    for x in ensemble_recommendations_f(UID):
        return f'ИД пользователя {UID} - рекомендованная шутка {x}', f'топ 10 шуток - {ensemble_recommendations_f(UID)}'
               

In [ ]:
final_func(1680)

('ИД пользователя 1680 - рекомендованная шутка 61',
 'топ 10 шуток - [61, 54, 53, 72, 76, 68, 35, 89, 50, 89]')